In [2]:
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn import metrics
data = pd.read_csv("WA_Fn-UseC_-Telco-Customer-Churn.csv")
data["TotalCharges"] = data["TotalCharges"].replace(r'\s+',0,regex = True)
data["TotalCharges"] = pd.to_numeric(data["TotalCharges"])
data["TotalCharges"] = data["TotalCharges"].replace(0,round(data["TotalCharges"].mean()))
def tenure(data):
    if data["tenure"] <= 12:
        return "0-12"
    elif (data["tenure"]>12) & (data["tenure"] <= 24):
        return "12-24"
    elif (data["tenure"]>24) & (data["tenure"] <= 36):
        return "24-36"
    elif (data["tenure"]>36) & (data["tenure"] <= 48):
        return "36-48"
    elif (data["tenure"]>48) & (data["tenure"] <= 60):
        return "48-60"
    elif (data["tenure"]>60):
        return "60--"
data["tengroup"] = data.apply(lambda data:tenure(data),axis = 1)
data = data.drop(["tenure","customerID"],axis = 1)
cols = ["OnlineSecurity","OnlineBackup","DeviceProtection","TechSupport","StreamingTV","StreamingMovies"]
for i in cols:
    data[i] = data[i].replace("No internet service","No")
data["MultipleLines"] = data["MultipleLines"].replace("No phone service","No")  
cat_cols   = data.nunique()[data.nunique() < 3].keys().tolist()
le = LabelEncoder()
for i in cat_cols:
    data[i] = le.fit_transform(data[i])
data = pd.get_dummies(data,columns=["InternetService","Contract","PaymentMethod","tengroup"],drop_first="bool")
std = StandardScaler()
num_cols = ["MonthlyCharges","TotalCharges"]
scaled_cols = ["scaledmonthlycharge","scaledtotalcharge"]
standard = std.fit_transform(data[num_cols])
standard = pd.DataFrame(standard,columns = scaled_cols)
frame = [standard,data]
datas = pd.concat(frame,axis = 1)
data = datas.drop(["MonthlyCharges","TotalCharges"],axis=1)
y = data["Churn"]
x = data.drop("Churn",axis = 1)
x_train,x_test,y_train,y_test = train_test_split(x,y,random_state = 1,test_size = .3)
rf = RandomForestClassifier(n_estimators=1100,max_depth=15,min_samples_split=20,
                            criterion= "gini",oob_score=True,random_state = 42,n_jobs = -1,min_samples_leaf=25)
rf.fit(x_train,y_train)
prediction = rf.predict(x_test)
print("accuracy_score",accuracy_score(y_test,prediction))
print("classification report:",metrics.classification_report(y_test,prediction))

accuracy_score 0.812588736393753
classification report:              precision    recall  f1-score   support

          0       0.85      0.91      0.88      1585
          1       0.66      0.52      0.58       528

avg / total       0.80      0.81      0.80      2113

